In [ ]:
import os
import re
import time
import math
import json
import torch
import random
import os.path
import torchtext
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.nn.functional as F




from math import pi
from math import sqrt
from glob import glob
from tqdm import tqdm
from torch import optim
from torchtext import vocab
from konlpy.tag import Mecab
from torchinfo import summary
from torchtext.vocab import vocab
from math import factorial as fact
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
#from torchsummary import summary as summary
from torchtext.data.metrics import bleu_score
from nltk.translate.bleu_score import sentence_bleu
from torchtext.vocab import build_vocab_from_iterator


%matplotlib inline


In [ ]:
hidden_size = 256
EPOCHS = 200
batch_size = 1024
num_layer = 3

BATCH_SIZE = 256


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.manual_seed(777)

if device == 'cuda':
    torch.cuda.manual_seed_all(777)


## 파일 확인

In [ ]:
question = sorted(glob('../embeddings/*44*'))


## 임베딩 벡터 로드 함수

In [ ]:
def load_embedding(name):
    if name == 'fast':
        que_emb= torchtext.vocab.Vectors(question[1])
        #equ_emb = torchtext.vocab.Vectors(equation[1])
    elif name == 'glove':
        que_emb = torchtext.vocab.Vectors(question[3])
        #equ_emb = torchtext.vocab.Vectors(equation[3])
    elif name == 'cbow':
        que_emb = torchtext.vocab.Vectors(question[0])
        #equ_emb = torchtext.vocab.Vectors(equation[0])
    else:
        que_emb = torchtext.vocab.Vectors(question[2])
        #equ_emb = torchtext.vocab.Vectors(equation[2])
    
    que_vocab = torchtext.vocab.vocab(que_emb.stoi, min_freq= 0, specials = ["<unk>", "<pad>", "<sos>", "<eos>"])
    #equ_vocab = torchtext.vocab.vocab(equ_emb.stoi, min_freq= 0)
    
    vector_dic = {'que': que_emb} #{'que': que_emb, 'equ': equ_emb} # dictionary 형태로 해당 단어에 대한 벡터 값들 저장
    
    vocab_dic = {'que': que_vocab} #{'que' : que_vocab, 'equ' : equ_vocab} # 위와 같은 형태로 단어 사전 저장
    
    for vocab in ['que']:
        a = torch.zeros(4,256, requires_grad = True)                                 ## 해당 스페션 토큰에 대한 임베딩 값을 만들어주기 위함
        vector_dic[vocab].vectors= torch.cat([a, vector_dic[vocab].vectors], dim=0) #              concat 시켜 기존 벡터들에 스페셜 토큰의 임베딩 벡터를 만들어줌
        vector_dic[vocab].stoi = dict(zip(vector_dic[vocab].stoi.keys(), map(lambda x:x[1]+4, vector_dic[vocab].stoi.items()))) # 스페셜 토큰이 추가 되었으니 기존 단어들의 인덱스를 뒤로 밀어줘야함
        for i, j in enumerate(["<unk>", "<pad>", "<sos>", "<eos>"]):
            vector_dic[vocab].stoi[j] = i                  # 사전훈련으로 임베딩 된 단어들은 special 토큰이 없기에 삽입, torchtext 상위 버전에서는 따로 함수  제공
        vocab_dic[vocab].set_default_index(0)                    #여기서 0번 인덱스는 <unk>를 의미함
    
    return vocab_dic, vector_dic

## 임베딩 로드

In [ ]:
pre_vocab ,pre_emb = load_embedding('fast')

In [ ]:
pre_vocab['que']['일까']

## 전처리

In [ ]:


with open('../data/train_44_type_150000.json', 'r', encoding = 'utf-8-sig') as json_:
    json_data = json.load(json_)

train_json= [[] for i in range(len(json_data))]

for i in json_data:
    for j in json_data[i]:
        train_json[int(i)].append(json_data[i][j])
        
        

with open('../data/val_44_type_30000.json', 'r', encoding = 'utf-8-sig') as json_:
    json_data = json.load(json_)

valid_json= [[] for i in range(len(json_data))]

for i in json_data:
    for j in json_data[i]:
        valid_json[int(i)].append(json_data[i][j])
        
with open('../data/test_44_type_30000.json', 'r', encoding = 'utf-8-sig') as json_:
    json_data = json.load(json_)

test_json= [[] for i in range(len(json_data))]

for i in json_data:
    for j in json_data[i]:
        test_json[int(i)].append(json_data[i][j])

## 문제, 수식 토크나이저

In [ ]:
def que_tokenizer(sentence, is_stopwords = False):
    '''
        Args:
            sentence (str) : 입력 문장
    
        Returns: 
            mecab.morphs(sentence) (list[str]) : 리스트에 형태소 단위로 분해됨.
    '''
    mecab = Mecab()
    stopwords = ['은','?','일까요','십시오','입니까','인가요',
             ',인지','한다면','가','다','에는','에서','기록', '순위',
             '앉아', '줄로', '.','시','오', '습니다','인지','한다','여라']
    
    if is_stopwords:
        a = mecab.morphs(sentence)
        a = [word for word in a if word not in stopwords]
        return a
    
    else:
        
        return mecab.morphs(sentence) ## 형태소 기준으로 분해

def yield_que_tokenizer(sentence, make_vocab = False):
    '''
        Args:
            sentence (str) : 입력 문장
    
        Returns: 
            mecab.morphs(sentence) (list[str]) : 리스트에 형태소 단위로 분해됨.
    '''
    stopwords = ['은','?','일까요','십시오','입니까','인가요',
             ',인지','한다면','가','다','에는','에서','기록', '순위',
             '앉아', '줄로', '.','시','오', '습니다','인지','한다','여라']

    mecab = Mecab()
    for i in sentence:
        a = mecab.morphs(i[0])
        a = [word for word in a if word not in stopwords]
        yield a   
      
                 

def yield_equ_tokenizer(sentence, make_vocab= False):
    '''
        Args:
            sentence (str) : 입력 문장
    
        Returns: 
            mecab.morphs(sentence) (list[str]) : 리스트에 형태소 단위로 분해됨.
    '''
    for i in sentence:
        equ = re.sub("([()])", r' \1 ', i[1])
        equ = re.sub("([\+\-\*\%\//\'\[\]\,\>\<])", r' \1 ',equ)
        yield equ.split()

def equ_tokenizer(sentence):
    '''
        Args:
            sentence (str) : 입력 문장
    
        Returns: 
            mecab.morphs(sentence) (list[str]) : 리스트에 형태소 단위로 분해됨.
    '''
    equ = re.sub("([()])", r' \1 ', sentence)
    equ = re.sub("([\+\-\*\%\//\'\[\]\,\>\<])", r' \1 ',equ)
    return equ.split()

## 단어장 생성

In [ ]:

equ_vocab = build_vocab_from_iterator(yield_equ_tokenizer(train_json, make_vocab=True), specials = ["<unk>", "<pad>","<sos>", "<eos>"], min_freq = 1)
equ_vocab.set_default_index(equ_vocab['<unk>'])


## 3. 데이터 로더

In [ ]:
def text_transform(sentence, is_que = False):
    
    '''
        문장을 인덱스로 변환하는 함수
        
        Args:
            sentence (str): 입력 문장.
        
        Returns:
            text2index (list[(int)]) : 문장을 토큰화 하고 해당 하는 토큰의 인덱스 반환.    
    '''
    if is_que == True:
        vocab = pre_vocab['que'] ## 수학문제 단어장
        text2index = [vocab[token] for token in que_tokenizer(sentence, True)] + [vocab['<eos>']] ## Stopword 설정
    else:
        vocab = equ_vocab ## 수식 단어장
        text2index = [vocab[token] for token in equ_tokenizer(sentence)] + [vocab['<eos>']]
        
    return text2index

In [ ]:
def max_length(df):
    '''
        Args:
            df (DataFrame): 데이터 프레임, column 0 은 question, 1은 equation
        
        Returns:
            que_length (int) : 문제를 토큰화 한 후 가장 긴 길이 반환
            equ_length (int) :  식을 토큰화 한 후 가장 긴 길이 반환.
    '''
    length = 0
    list_que = []
    for i in range(len(df)):
        list_que.append(len(que_tokenizer(df[i][0], True)))
    que_length = max(list_que)
    que_length += 1 ## eos 까지 붙는걸 계산
    return que_length #,equ_length


In [ ]:

def new_collate_batch(batch):
    '''
        배치로 묶어주는 함수 DataLoader를 쓸 때 사용
    '''
    src_list, target_list, ans_list = [],[], []
    for que, equ, ans in batch:
        processed_src = torch.tensor(text_transform(que, is_que = True))
        processed_tar= torch.tensor(text_transform(equ))
        src_list.append(processed_src)
        target_list.append(processed_tar)
        ans_list.append(ans)
        
    pad_src_list = pad_sequence(src_list, padding_value = 1.0)
    pad_target_list = pad_sequence(target_list, padding_value = 1.0) ## [sequence, batch] 형태로 만들어주는 pad_sequence, padding은 <pad>의 인덱스 번호 1로 설정
    return pad_src_list, pad_target_list, ans_list

## 데이터 로더 생성

In [ ]:
train_loader = DataLoader(train_json, batch_size = batch_size, shuffle = True, collate_fn = new_collate_batch, drop_last = True)
valid_loader = DataLoader(valid_json, batch_size = batch_size, shuffle = True, collate_fn = new_collate_batch, drop_last = True)
test_loader = DataLoader(test_json, batch_size = batch_size, shuffle = False, collate_fn = new_collate_batch, drop_last = True)

## 모델 설계

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_vocab, hidden_size ,batch_size ,num_layer, pretrain = False):
        '''
            input_vocab : 입력 언어 단어장 크기
            hidden_size : 임베딩 층 히든 크기
            batch_size (int) : 배치 크기
            num_layer (int) : RNN 레이어 층 개수
            pretrain (bool) : 사전학습 시킨 단어 임베딩을 사용할 건지 안할 건지
        '''
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.batch = batch_size
        self.n_layer = num_layer
        self.gru = nn.GRU(hidden_size, hidden_size, num_layers = self.n_layer)
        self.pretrain = pretrain
        
        
        if pretrain:
            self.embedding = nn.Embedding.from_pretrained(pre_emb['que'].vectors, padding_idx=1, freeze = False)
        else:
            self.embedding = nn.Embedding(input_vocab, hidden_size, padding_idx = 1)
            
        

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1,self.batch,-1) ## [sequence_length, batch, hidden_size]
        output = embedded
        output, hidden = self.gru(output, hidden)  
        return output, hidden # output = [sequence_length, batch, hidden_size] , hidden_out = [D*num_layer, batch, hidden_size]
    
    def initHidden(self):
        return torch.zeros(self.n_layer, self.batch, self.hidden_size, device = device) ## hidden_in [D*num_layer, batch, hidden_size]  

In [ ]:
class LuongAttnDecoderRNN(nn.Module):
    '''
            Args:
                output_size (int) : 출력 언어 단어 사전 크기, 여기선 식 임베딩 단어장 크기
                hidden_size (int) : embedding layer hidden size
                max_length (int) : 문장형 문제 중 문장 중 가장 긴 문장 값
                batch_size (int) : 배치 크기
                num_layer (int) : RNN 레이어 층 개수
                dropout_p (float) : 드랍아웃 확률
                pretrain (bool) : 사전학습 시킨 단어 임베딩을 사용할 건지 안할 건지
            
        '''
    
    def __init__(self, output_size, hidden_size, max_length, batch_size, num_layer, dropout_p = 0.4, pretrain = False):
        super(LuongAttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.batch = batch_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length
        self.n_layer = num_layer
        
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length) ##concat, 2개의 히든을 받고 문장의 최대 길이 만큼 반환
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.fc = nn.Linear(hidden_size, hidden_size, bias=False)
    
        self.gru = nn.GRU(self.hidden_size, self.hidden_size, num_layers = self.n_layer)
        self.classifier = nn.Linear(self.hidden_size * 2, self.output_size)
                        
        if pretrain:
            self.embedding = nn.Embedding.from_pretrained(pre_emb['equ'].vectors, padding_idx=1, freeze = False) #eng
        else:
            self.embedding = nn.Embedding(output_size, hidden_size, padding_idx = 1)
            
        
        

    def forward(self, input, hidden, encoder_outputs):
        '''
        input: input length [해당 문장 길이, 배치]
        hidden: encoder hidden [레이어, 배치, 히든]
        encoder_outputs : 어텐션 계산을 위한 인코더 히든값 encoder_outputs [배치, 최대 길이, 히든]
            
        '''
        embedded = self.embedding(input).view(1,self.batch,-1) ## [1,128,hidden] (input_length, batch_size, hidden_size) 
        embedded = self.dropout(embedded)
        
        gru_out, hidden = self.gru(embedded, hidden) ## gru_out = [1, batch , hidden]
        
        permute_gru = gru_out.permute(1,0,2) # [batch,1,hidden]
        
        out = self.fc(permute_gru) # out = [batch,hidden]
        
        #out = out.unsqueeze(1)  # out = [batch,1,hidden]
        
        permute_outputs = encoder_outputs.permute(0,2,1) # length 와 hidden 위치를 바꿔줌
         
        alignment_scores= out.bmm(permute_outputs) #[batch, 1 , hidden] @ [batch ,hidden, length] = [batch, 1 , length] => attn score
        
        attn_weights = F.softmax(alignment_scores, dim=2) # [batch,1, length]
        
        context_vector = torch.bmm(attn_weights,encoder_outputs) ## [batch, 1, hidden]
        
        output = torch.cat((permute_gru, context_vector),dim = 2) #[batch, 1 , hidden * 2]
        
        output = output.squeeze() # batch, hidden * 2
        
        output = F.log_softmax(self.classifier(output), dim=1)
        
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(self.n_layer, self.batch, self.hidden_size, device=device) #layer

In [ ]:
teacher_forcing_ratio = 0.3

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length):
    '''
        Args:
            input_tensor (tensor) : [시퀀스, 배치]
            target_tensor (tensor): [시퀀스, 배치]
            encoder (Module) : 인코더 모델
            decoder (Module) : 디코더 모델
            encoder_optimizer (optimizer)  
            decoder_optimizer (optimizer) 
            criterion (Loss)
            max_length : 인코더 입력 문장 최대 길이
            
        Returns:
            loss.item() / target_length (float) : target_length 만큼 loss 값을 더해 줘서 length로 나눠줌 
    '''
    
    encoder_hidden = encoder.initHidden() #[layer, batch, hidden_size]
    
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    
    input_length = input_tensor.size(0)  
    target_length = target_tensor.size(0)
    
    encoder_outputs = torch.zeros(input_tensor.size(1), max_length, encoder.hidden_size, device = device) ## 어텐션 용, [batch, sequence_length, hidden]
    
    loss = 0
    
    for ei in range(input_length):  #한 글자씩 들어감  #### 한 개 씩 말고 그냥 배치 전체로 돌리는것도 고려
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden) ## [1, 배치, 히든] 
        encoder_outputs[:, ei, :] = encoder_output[0,:] #[128, ei, 256] 
    

        
    if encoder.pretrain:
        q_vocab = pre_vocab['que']
        e_vocab = equ_vocab
    else:
        q_vocab = que_vocab
        e_vocab = equ_vocab
    
    
    decoder_input = torch.tensor([[q_vocab['<sos>'] for i in range(input_tensor.size(1))]], device = device) 
    ##[문장 길이,배치] #문장의 시작을 알리는 index를 배치 크기 만큼 입력

    decoder_hidden = encoder_hidden #input_length에서 마지막 단어까지 거친 히든 [layer, batch, hidden ]
    
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    
    if use_teacher_forcing:
        for di in range(target_length): #타겟 문장 전체 반복
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs) 
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]
    else:
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()
            loss += criterion(decoder_output, target_tensor[di])
            for i in range(input_tensor.size(1)):
                if topi[i].item() == q_vocab['<eos>']:
                    continue
                    
    
    loss.backward()
    
    encoder_optimizer.step()
    decoder_optimizer.step()
    
    return loss.item() / target_length
                

In [ ]:
def new_evaluate(encoder, decoder, data_loader, que_len, criterion):
    
    total_loss = 0
    total_bleu = 0
    acc = 0
    prefect_predict = 0
    
    encoder.eval()
    decoder.eval()    
                
    if encoder.pretrain:
        q_vocab = pre_vocab['que']
        e_vocab = equ_vocab #e_vocab = pre_vocab['equ']
    else:
        q_vocab = que_vocab
        e_vocab = equ_vocab
    

    with torch.no_grad():
        for src, tgt, ans in data_loader:
            
            src = src.to(device)
            tgt = tgt.to(device)
            
            ans = list(map(float, np.array(ans)))

            
            input_length = src.size(0)
            target_length = tgt.size(0)
            encoder_hidden = encoder.initHidden()
            encoder_outputs = torch.zeros(src.size(1), que_len, encoder.hidden_size, device = device)

            for ei in range(input_length):
                encoder_output, encoder_hidden = encoder(src[ei], encoder_hidden)
                encoder_outputs[:, ei, :] = encoder_output[0, :]
            
            decoded_words = [[] for i in range(src.size(1))]
            
            decoder_input = torch.tensor([[q_vocab['<sos>'] for i in range(src.size(1))]], device = device) ## 처음 시작은 <sos> 토큰

            loss = 0.0
        
            decoder_hidden = encoder_hidden
        
            decoded_word = []
            
            squeeze_index = []

            
            for di in range(target_length): 
                decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs) #if attention, encoder_outputs 추가
                topv, topi = decoder_output.topk(1) ## topi = [batch_size, 1]
                
                loss += criterion(decoder_output, tgt[di])
                
                squeeze_index.append(topi.squeeze()) ## topi.squeeze() = [batch_size, ]  squeeze_index = [[batch],[batch],[batch] ''']
                decoder_input= topi.squeeze().detach()
            
            stack_squeeze = torch.stack(squeeze_index) #[length, batch]
            
            total_loss += (loss / target_length)           
            
            for batch_idx in range(src.size(1)):         ## 배치의 각 문장마다 돌면서 <eos> 토큰을 만나면 문장을 끊도록
                a = stack_squeeze[:, batch_idx]
                subwords = []
                for j, k in enumerate(a):
                    if a[j] == 3:
                        subwords.append("<eos>")
                        decoded_word.append(subwords)
                        break
                    else:
                        subwords.append(e_vocab.lookup_token(a[j])) ##eng_vocab
                        if len(subwords) == (target_length -1):
                            subwords.append("<eos>")
                            decoded_word.append(subwords)
                            break
 
            
            real = [(" ".join(e_vocab.lookup_tokens(tgt.T.tolist()[r])).replace("<pad>", "")).split() for r in range(tgt.size(1))] 
            ## replace('<eos>', "").split()


            pre_bleu = 0 ## 배치의 bleu score
            
            

            for pred_idx in range(src.size(1)):
                equation = (' '.join(decoded_word[pred_idx])).replace('<eos>',"")
                try:
                    predict_ans = eval(equation.replace('@', '//')) ## 생성된 식을 eval로 계산
                    predict_ans = float('%.02f' % predict_ans)
                except (SyntaxError, TypeError, ZeroDivisionError,ValueError, OverflowError):              ## 잘못된 식 생성시 에러 발생
                    continue
                    
                if predict_ans == ans[pred_idx]:
                    acc += 1

                bleu_score = sentence_bleu([real[pred_idx]], decoded_word[pred_idx], smoothing_function=None, weights = (0.25,0.25,0.25,0.25))
                
                if bleu_score == 1.0:
                    prefect_predict += 1 
                pre_bleu += bleu_score
                
                print(f"sentence {pred_idx}" )
                print("문장형 문제 =", (' '.join(q_vocab.lookup_tokens(src.T.tolist()[pred_idx]))).replace("<pad>", "").replace("<eos>",""))
                print(f"실제 수식  = ", (' '.join(real[pred_idx])))
                print("실제 정답", ans[pred_idx])
                print(f"예측 수식  = ", " ".join(decoded_word[pred_idx]))
                print("예측 정답", predict_ans)

                
            
            pre_bleu = pre_bleu / src.size(1)
            total_bleu += pre_bleu
            
        print(f"mean BLEU score : {total_bleu / len(data_loader)}, prefect_prediction : {prefect_predict} sentence, Accuracy : {acc / len(data_loader.dataset)}")
    
    
    #return total_bleu/ len(data_loader), total_loss.item() / len(data_loader) , acc / len(data_loader.dataset)


In [ ]:
def trainIters(encoder, decoder, train_loader, max_length):
    
    encoder.train()
    decoder.train()
    plot_losses = []
    print_loss_total = 0
    plot_loss_total = 0

    
    for src, tgt, _ in train_loader:
        src = src.to(device)
        tgt = tgt.to(device)
        loss = train(src, tgt, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length)
        print_loss_total += loss 
        
    print_loss_avg = print_loss_total / len(train_loader)
    
    return print_loss_avg
    
          

In [ ]:
def new_greedy(encoder, decoder, data_loader, que_len, criterion):
    
    total_loss = 0
    total_bleu = 0
    acc = 0
    prefect_predict = 0
    
    encoder.eval()
    decoder.eval()    
                
    if encoder.pretrain:
        q_vocab = pre_vocab['que']
        e_vocab = equ_vocab #e_vocab = pre_vocab['equ']
    else:
        q_vocab = que_vocab
        e_vocab = equ_vocab
    

    with torch.no_grad():
        for src, tgt, ans in data_loader:
            
            src = src.to(device)
            tgt = tgt.to(device)
            
            ans = list(map(float, np.array(ans)))

            
            input_length = src.size(0)
            target_length = tgt.size(0)
            encoder_hidden = encoder.initHidden()
            encoder_outputs = torch.zeros(src.size(1), que_len, encoder.hidden_size, device = device)

            for ei in range(input_length):
                encoder_output, encoder_hidden = encoder(src[ei], encoder_hidden)
                encoder_outputs[:, ei, :] = encoder_output[0, :]
            
            decoded_words = [[] for i in range(src.size(1))]
            
            decoder_input = torch.tensor([[q_vocab['<sos>'] for i in range(src.size(1))]], device = device) ## 처음 시작은 <sos> 토큰

            loss = 0.0
        
            decoder_hidden = encoder_hidden
        
            decoded_word = []
            
            squeeze_index = []

            
            for di in range(target_length): 
                decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs) #if attention, encoder_outputs 추가
                topv, topi = decoder_output.topk(1) ## topi = [batch_size, 1]
                
                loss += criterion(decoder_output, tgt[di])
                
                squeeze_index.append(topi.squeeze()) ## topi.squeeze() = [batch_size, ]  squeeze_index = [[batch],[batch],[batch] ''']
                decoder_input= topi.squeeze().detach()
            
            stack_squeeze = torch.stack(squeeze_index) #[length, batch]
            
            total_loss += (loss / target_length)           
            
            for batch_idx in range(src.size(1)):         ## 배치의 각 문장마다 돌면서 <eos> 토큰을 만나면 문장을 끊도록
                a = stack_squeeze[:, batch_idx]
                subwords = []
                for j, k in enumerate(a):
                    if a[j] == 3:
                        subwords.append("<eos>")
                        decoded_word.append(subwords)
                        break
                    else:
                        subwords.append(e_vocab.lookup_token(a[j])) ##eng_vocab
                        if len(subwords) == (target_length -1):
                            subwords.append("<eos>")
                            decoded_word.append(subwords)
                            break
            
            
            real = [(" ".join(e_vocab.lookup_tokens(tgt.T.tolist()[r])).replace("<pad>", "")).split() for r in range(tgt.size(1))] 
            ## replace('<eos>', "").split()


            pre_bleu = 0 ## 배치의 bleu score
            
            
            for pred_idx in range(src.size(1)):
                equation = (' '.join(decoded_word[pred_idx])).replace('<eos>',"")
                try:
                    predict_ans = eval(equation.replace('@', '//')) ## 생성된 식을 eval로 계산
                    predict_ans = float('%.02f' % predict_ans)
                except (SyntaxError, TypeError, ZeroDivisionError,ValueError):              ## 잘못된 식 생성시 에러 발생
                    continue
                    
                if predict_ans == ans[pred_idx]:
                    acc += 1
                    
                bleu_score = sentence_bleu([real[pred_idx]], decoded_word[pred_idx], smoothing_function=None, weights = (0.25,0.25,0.25,0.25))
                
                if bleu_score == 1.0:
                    prefect_predict += 1 
                pre_bleu += bleu_score
                
            
            pre_bleu = pre_bleu / src.size(1)
            total_bleu += pre_bleu
            
    
    
    return total_bleu/ len(data_loader), total_loss.item() / len(data_loader) , acc / len(data_loader.dataset)


In [ ]:
que_len= max_length(train_json)
valid_len = max_length(valid_json)
test_len = 0
que_len =max(que_len, valid_len, test_len)

In [ ]:
que_len = 98

## 모델 선언

In [ ]:
encoder = EncoderRNN(len(pre_vocab['que']), hidden_size ,batch_size ,num_layer ,True).to(device)
attn_decoder = LuongAttnDecoderRNN(len(equ_vocab), hidden_size, que_len, batch_size, num_layer, False).to(device)


In [ ]:
encoder_optimizer = optim.Adam(encoder.parameters(), lr = 5e-4)
decoder_optimizer = optim.Adam(attn_decoder.parameters(), lr = 5e-4)
criterion = nn.NLLLoss(ignore_index = 1)

In [ ]:
criterion = nn.NLLLoss(ignore_index = 1)

In [ ]:
t = open('attn_loss/Lu_train_losses','w')
b = open('attn_loss/Lu_bleu_score','w')
v = open('attn_loss/Lu_valid_losses','w')
a = open('attn_loss/Lu_accuracy','w')
t.close()
b.close()
v.close()
a.close()


## 학습

In [ ]:


def start_train(encoder, attn_decoder, directory):
    best_loss = None
    prev_loss= 99999999
    

    
    i = 0
    criterion = nn.NLLLoss(ignore_index = 1)
    for epoch in range(1, 400+1):
        os.chdir('/data/kmkim/MWP_model')
        t = open('attn_loss/Lu_train_losses','a')
        b = open('attn_loss/Lu_bleu_score','a')
        v = open('attn_loss/Lu_valid_losses','a')
        a = open('attn_loss/Lu_accuracy','a')
    
        try:                
            train_time = time.time()
            train_loss = trainIters(encoder, attn_decoder, train_loader, que_len) ##trainiter 과 train 연관 하여서 시간이랑 loss 계산 완벽하게
            train_end_time = time.time()
            #train_bleu, train_loss, train_acc = new_greedy(encoder, attn_decoder, train_loader, que_len, criterion)
            val_bleu ,val_loss, val_acc = new_greedy(encoder, attn_decoder, valid_loader, que_len, criterion)


            t.write(str(train_loss) + '\n')
            b.write(str(val_bleu) + '\n')
            v.write(str(val_loss) + '\n')
            a.write(str(val_acc) + '\n')
        
        finally:
            t.close()
            b.close()
            v.close()
            a.close()
            
        valid_end_time = time.time()

        print(f"Epoch: {epoch}, train_time = {(train_end_time - train_time):.3f}s, train_loss = {train_loss:.4f}, \
        validation = {val_loss:.4f}, val_bleu = {val_bleu:.4f}, val_acc ={val_acc:.4f}, valid_time = {(valid_end_time - train_end_time):.3f}s") 
        

        if not best_loss or val_loss < best_loss:
            os.chdir('/data/kmkim/MWP/new_problem')
            if not os.path.isdir(directory):
                os.makedirs(directory)
            torch.save(encoder.state_dict(), './'+ directory +'/attn_encoder.pt') 
            torch.save(attn_decoder.state_dict(), './'+directory+'/attn_decoder.pt')
            print('model save, directory:', directory)

            best_loss = val_loss

            if prev_loss < val_loss:
                i += 1
                if i > 5:
                    print("patient over 5")
                    break
            else:
                i = 0

            prev_loss = val_loss
            
    
 
    

In [ ]:
start_train(encoder, attn_decoder, 'mwp_Lu_attn_44_type')

In [ ]:
encoder_T = EncoderRNN(len(pre_vocab['que']), hidden_size ,batch_size ,num_layer, True).to(device)
attn_decoder_T = LuongAttnDecoderRNN(len(equ_vocab), hidden_size, que_len, batch_size, num_layer, False).to(device)
encoder_T.load_state_dict(torch.load('../saved_weights/attn_encoder.pt'))
attn_decoder_T.load_state_dict(torch.load('../saved_weights/attn_decoder.pt'))

In [ ]:

test_csv = pd.read_csv("../data/math_test_1000" , header=None)


def data_format(df, is_stopwords = False):
    '''
        Args:
            df (DataFrame) : 데이터프레임 csv
        
        Returns:
            list_ (list): 데이터를 받아서 ['문장', '라벨'] 로 연결
        
            ex) list_= [['민혁이가 잰 직사각형의 둘레는 48m이고, 직사각형의 세로는 가로 길이의 2배 입니다. 세로는 몇 m인지 구하세요.', '((48 / 2) / (2 + 1)) * 2'],
    '''
    list_ = [] 
    for i in range(len(df[0])):
        list_.append([df[0][i]])
    for i in range(len(df[0])):    
        list_[i].append(df[1][i])
    for i in range(len(df[0])):
        list_[i].append(df[2][i])
        
    
    return list_

test_pairs = data_format(test_csv)

real_loader = DataLoader(test_pairs, batch_size = 100, shuffle = False, collate_fn = new_collate_batch, drop_last = True)


## 모델 평가

In [ ]:
from torchmetrics.text import CharErrorRate
from torchmetrics.text import WordErrorRate
from torchmetrics.text.rouge import ROUGEScore


wer = WordErrorRate()
cer = CharErrorRate()
rouge = ROUGEScore(rouge_keys = ('rougeL'))['rougeL_fmeasure']

In [ ]:
def new_new_evaluate(encoder, decoder, data_loader, que_len):
    
    total_bleu = 0
    bleu_1gram = 0
    bleu_2gram = 0
    bleu_3gram = 0
    total_cer = 0
    total_rouge = 0
    acc = 0
    
    ## 이하는 나중에 지워도 됨
    correct= 0
    new_bleu_acc = 0
    data_index = []

    encoder.eval()
    decoder.eval()    
                
    if encoder.pretrain:
        q_vocab = pre_vocab['que']
        e_vocab = equ_vocab #e_vocab = pre_vocab['equ']
    else:
        q_vocab = que_vocab
        e_vocab = equ_vocab
    

    with torch.no_grad():
        for loader_idx, (src, tgt, ans) in enumerate(tqdm(data_loader)):
            
            src = src.to(device)
            tgt = tgt.to(device)
            
            ans = list(map(float, np.array(ans)))

            
            input_length = src.size(0)
            target_length = tgt.size(0)
            encoder_hidden = encoder.initHidden()
            encoder_outputs = torch.zeros(src.size(1), que_len, encoder.hidden_size, device = device)

            for ei in range(input_length):
                encoder_output, encoder_hidden = encoder(src[ei], encoder_hidden)
                encoder_outputs[:, ei, :] = encoder_output[0, :]
            
            decoded_words = [[] for i in range(src.size(1))]
            
            decoder_input = torch.tensor([[q_vocab['<sos>'] for i in range(src.size(1))]], device = device) ## 처음 시작은 <sos> 토큰

        
            decoder_hidden = encoder_hidden
        
            decoded_word = []
            
            squeeze_index = []

            
            for di in range(target_length): 
                decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs) #if attention, encoder_outputs 추가
                topv, topi = decoder_output.topk(1) ## topi = [batch_size, 1]
                
                squeeze_index.append(topi.squeeze()) ## topi.squeeze() = [batch_size, ]  squeeze_index = [[batch],[batch],[batch] ''']
                decoder_input= topi.squeeze().detach()
            
            stack_squeeze = torch.stack(squeeze_index) #[length, batch]
                        
            for batch_idx in range(src.size(1)):         ## 배치의 각 문장마다 돌면서 <eos> 토큰을 만나면 문장을 끊도록
                a = stack_squeeze[:, batch_idx]
                subwords = []
                for j, k in enumerate(a):
                    if a[j] == 3:
                        subwords.append("<eos>")
                        decoded_word.append(subwords)
                        break
                    else:
                        subwords.append(e_vocab.lookup_token(a[j])) ##eng_vocab
                        if len(subwords) == (target_length -1):
                            subwords.append("<eos>")
                            decoded_word.append(subwords)
                            break
 
            
            real = [(" ".join(e_vocab.lookup_tokens(tgt.T.tolist()[r])).replace("<pad>", "")).split() for r in range(tgt.size(1))] 
            ## replace('<eos>', "").split()
    

            for pred_idx in range(src.size(1)):
                equation = (' '.join(decoded_word[pred_idx])).replace('<eos>',"")
                real_equ = (" ".join(equ_vocab.lookup_tokens(tgt.T.tolist()[pred_idx]))).replace("<sos>", "").replace("<eos>","").replace("<pad>", "")

                try:
                    predict_ans = eval(equation.replace('@', '//')) ## 생성된 식을 eval로 계산
                    predict_ans = float('%.02f' % predict_ans)
                except (SyntaxError, TypeError, ZeroDivisionError,ValueError, OverflowError):              ## 잘못된 식 생성시 에러 발생
                    continue
                    
                if predict_ans == ans[pred_idx]:
                    acc += 1
                
                
                real = real_equ.split()
                real.extend(["<eos>"])
                
                
                total_bleu += sentence_bleu([real], decoded_word[pred_idx]) # SmoothingFunction().method6
                bleu_1gram += sentence_bleu([real], decoded_word[pred_idx], weights = [(1.0)])
                bleu_2gram += sentence_bleu([real], decoded_word[pred_idx], weights = [(0.5,0.5)])
                bleu_3gram += sentence_bleu([real], decoded_word[pred_idx], weights = [(0.333,0.333,0.334)])                

                total_cer += cer([" ".join(decoded_word[pred_idx])],[' '.join(real)]).item()
                total_rouge += rouge([" ".join(decoded_word[pred_idx])], [' '.join(real)]).item()
                
                
                #if ans[pred_idx] == predict_ans:
                if (loader_idx == len(data_loader)-1) and (batch_idx == src.size(1) - 1):
                    
                    print(f"sentence {pred_idx}" )
                    print("문장형 문제 =", (' '.join(pre_vocab['que'].lookup_tokens(src.T.tolist()[pred_idx]))).replace("<pad>", "").replace("<eos>",""))
                    print(f"실제 수식  = ", (' '.join(real)))
                    print("실제 정답", ans[pred_idx])
                    print(f"예측 수식  = ", " ".join(decoded_word[pred_idx]))
                    print("예측 정답", predict_ans)
                
            

    #return total_bleu/ len(data_loader), total_loss.item() / len(data_loader) , acc / len(data_loader.dataset)
    return total_bleu / (len(data_loader) * batch_size), acc / (len(data_loader) * batch_size), bleu_1gram / (len(data_loader) * batch_size), bleu_2gram / (len(data_loader) * batch_size), bleu_3gram / (len(data_loader) * batch_size), total_cer / (len(data_loader) * batch_size),total_rouge / (len(data_loader) * batch_size)

In [ ]:
new_new_evaluate(encoder_T, attn_decoder_T, test_loader, que_len)

In [ ]:
def new_new_test(encoder, decoder, data_loader, que_len):
    
    total_bleu = 0
    bleu_1gram = 0
    bleu_2gram = 0
    bleu_3gram = 0
    total_cer = 0
    total_rouge = 0
    error_count = 0
    acc = 0
    
    ## 이하는 나중에 지워도 됨
    correct= 0
    new_bleu_acc = 0
    data_index = []

    encoder.eval()
    decoder.eval()    
                
    if encoder.pretrain:
        q_vocab = pre_vocab['que']
        e_vocab = equ_vocab #e_vocab = pre_vocab['equ']
    else:
        q_vocab = que_vocab
        e_vocab = equ_vocab
    

    with torch.no_grad():
        for loader_idx, (src, tgt, ans) in enumerate(tqdm(data_loader)):
            
            src = src.to(device)
            tgt = tgt.to(device)
            
            ans = list(map(float, np.array(ans)))

            
            input_length = src.size(0)
            target_length = tgt.size(0)
            encoder_hidden = encoder.initHidden()
            encoder_outputs = torch.zeros(src.size(1), que_len, encoder.hidden_size, device = device)

            for ei in range(input_length):
                encoder_output, encoder_hidden = encoder(src[ei], encoder_hidden)
                encoder_outputs[:, ei, :] = encoder_output[0, :]
            
            decoded_words = [[] for i in range(src.size(1))]
            
            decoder_input = torch.tensor([[q_vocab['<sos>'] for i in range(src.size(1))]], device = device) ## 처음 시작은 <sos> 토큰

        
            decoder_hidden = encoder_hidden
        
            decoded_word = []
            
            squeeze_index = []

            
            for di in range(target_length): 
                decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs) #if attention, encoder_outputs 추가
                topv, topi = decoder_output.topk(1) ## topi = [batch_size, 1]
                
                squeeze_index.append(topi.squeeze()) ## topi.squeeze() = [batch_size, ]  squeeze_index = [[batch],[batch],[batch] ''']
                decoder_input= topi.squeeze().detach()
            
            stack_squeeze = torch.stack(squeeze_index) #[length, batch]
                        
            for batch_idx in range(src.size(1)):         ## 배치의 각 문장마다 돌면서 <eos> 토큰을 만나면 문장을 끊도록
                a = stack_squeeze[:, batch_idx]
                subwords = []
                for j, k in enumerate(a):
                    if a[j] == 3:
                        subwords.append("<eos>")
                        decoded_word.append(subwords)
                        break
                    else:
                        subwords.append(e_vocab.lookup_token(a[j])) ##eng_vocab
                        if len(subwords) == (target_length -1):
                            subwords.append("<eos>")
                            decoded_word.append(subwords)
                            break
 
            
            real = [(" ".join(e_vocab.lookup_tokens(tgt.T.tolist()[r])).replace("<pad>", "")).split() for r in range(tgt.size(1))] 
            ## replace('<eos>', "").split()
    

            for pred_idx in range(src.size(1)):
                equation = (' '.join(decoded_word[pred_idx])).replace('<eos>',"")
                real_equ = (" ".join(equ_vocab.lookup_tokens(tgt.T.tolist()[pred_idx]))).replace("<sos>", "").replace("<eos>","").replace("<pad>", "")

                try:
                    predict_ans = eval(equation.replace('@', '//')) ## 생성된 식을 eval로 계산
                    predict_ans = float('%.02f' % predict_ans)
                except (SyntaxError, TypeError, ZeroDivisionError,ValueError, OverflowError):              ## 잘못된 식 생성시 에러 발생
                    continue

                real = real_equ.split()
                real.extend(["<eos>"])
                
                
                if ans[pred_idx] == predict_ans:
                    correct += 1
                    print("***********정답******************")
                    print(f"데이터 몇번째 :{loader_idx * 10 + pred_idx +1} ") ## 10 = real_dataloader batch size
                    data_index.append(loader_idx * 10 + pred_idx +1)
                    print("문장형 문제 =", (' '.join(pre_vocab['que'].lookup_tokens(src.T.tolist()[pred_idx]))).replace("<pad>", "").replace("<eos>",""))
                    print(f"실제 수식  = ", (' '.join(real)))
                    print(f"예측 수식  = ", " ".join(decoded_word[pred_idx]))
                    print("실제 정답", ans[pred_idx])
                    print("예측 정답", predict_ans)
                    bleu_score_acc = sentence_bleu([real], decoded_word[pred_idx],smoothing_function =None)
                    
                    bleu_1gram += sentence_bleu([real], decoded_word[pred_idx], weights = [(1.0)])
                    bleu_2gram += sentence_bleu([real], decoded_word[pred_idx], weights = [(0.5,0.5)])
                    bleu_3gram += sentence_bleu([real], decoded_word[pred_idx], weights = [(0.333,0.333,0.334)])
                    total_cer += wer([" ".join(decoded_word[pred_idx])],[' '.join(real)]).item()
                    total_rouge += rouge([" ".join(decoded_word[pred_idx])], [' '.join(real)]).item()                  
                    #total_cer += cer(real, decoded_words[pred_idx]).item()
                    new_bleu_acc += bleu_score_acc
                    
                    print("*********************************")
       
                else:
                    if error_count < 247:
                        print("^^^^^^^^^^오답^^^^^^^^^^^^^^^^^^^^^^^^^")
                        error_count += 1
                        print("문장형 문제 =", (' '.join(pre_vocab['que'].lookup_tokens(src.T.tolist()[pred_idx]))).replace("<pad>", "").replace("<eos>",""))
                        print(f"실제 수식  = ", (' '.join(real)))
                        print(f"예측 수식  = ", " ".join(decoded_word[pred_idx]))
                        print("실제 정답", ans[pred_idx])
                        print("예측 정답", predict_ans)
                        bleu_score_acc = sentence_bleu([real], decoded_word[pred_idx],smoothing_function =None)

                        bleu_1gram += sentence_bleu([real], decoded_word[pred_idx], weights = [(1.0)])
                        bleu_2gram += sentence_bleu([real], decoded_word[pred_idx], weights = [(0.5,0.5)])
                        bleu_3gram += sentence_bleu([real], decoded_word[pred_idx], weights = [(0.333,0.333,0.334)])
                        total_cer += wer([" ".join(decoded_word[pred_idx])],[' '.join(real)]).item()
                        total_rouge += rouge([" ".join(decoded_word[pred_idx])], [' '.join(real)]).item()
                        new_bleu_acc += bleu_score_acc
    
        print(f"total 300/ correct:{correct/311} ")
        print(f"correct : {correct}")
        print(f"data index",data_index)
        print(f"total 300 bleu :{new_bleu_acc/ 311}")
        print(f"total 300 bleu_1 :{bleu_1gram/ 311}")
        print(f"total 300 bleu_2 :{bleu_2gram/ 311}")
        print(f"total 300 bleu_3:{bleu_3gram/ 311}")
        print(f"total cer :{total_cer}")
        print(f"total 300 cer :{total_cer/ 311}")
        print(f"total 300 rouge :{total_rouge/ 311}")

        print(f"error count :{error_count}")
        

                    
                    
                

             
        
    return

## 실제 데이터 세트로 평가

In [ ]:
encoder_T = EncoderRNN(len(pre_vocab['que']), hidden_size ,100,num_layer, True).to(device)
attn_decoder_T = LuongAttnDecoderRNN(len(equ_vocab), hidden_size, que_len, 100, num_layer, False).to(device)
encoder_T.load_state_dict(torch.load('../saved_weights/attn_encoder.pt'))
attn_decoder_T.load_state_dict(torch.load('../saved_weights/attn_decoder.pt'))

In [ ]:
new_new_test(encoder_T, attn_decoder_T, real_loader, que_len)